In [2]:
from typing import Optional, List, Tuple
import os
import time
from collections import defaultdict

import botorch
import gpytorch
import numpy as np
import torch
from torch import Tensor
from botorch.acquisition.objective import LearnedObjective
from botorch.acquisition.preference import AnalyticExpectedUtilityOfBestOption
from botorch.fit import fit_gpytorch_mll
from botorch.models.multitask import KroneckerMultiTaskGP
from botorch.models.pairwise_gp import (PairwiseGP,
                                        PairwiseLaplaceMarginalLogLikelihood)
from botorch.models.transforms.input import (InputTransform, ChainedInputTransform,
                                             FilterFeatures, Normalize)
from botorch.models.transforms.outcome import (OutcomeTransform, ChainedOutcomeTransform,
                                               Standardize)
from botorch.optim.optimize import optimize_acqf
from botorch.posteriors import Posterior, TransformedPosterior

from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.test_functions.synthetic import SyntheticTestFunction
from botorch.utils.sampling import draw_sobol_samples
from gpytorch.mlls.exact_marginal_log_likelihood import \
    ExactMarginalLogLikelihood


In [3]:
# Simulator for the shapes problem

def swap(a,b):
    tmp = a
    a=b
    b=tmp

    return a, b

class Image(SyntheticTestFunction):
    r"""
    Class for generating rectangle images
    """
    dim = 4
    _bounds = torch.tensor([[0., 1.], [0., 1.], [0., 1.], [0.,1.]])

    def __init__(self, num_pixels: int = 16):
        super().__init__()
        self.num_pixels = num_pixels
        self.pixel_size = 1 / self.num_pixels
        self.outcome_dim = num_pixels ** 2
    
    def evaluate_true(self, X):
        r"""
        Args:
            X: 4-dimensional input
        Returns:
            Y: 256-dimensional array representing 16x16 images
        """

        # map real values in X to integer indices of pixels
        pixel_idcs = torch.div(X, self.pixel_size, rounding_mode="floor")

        Y = torch.zeros((*X.shape[:-1], self.num_pixels**2))

        for sample_idx in range(X.shape[-2]): # TODO: there could be shape erros

            row_start, col_start, row_end, col_end = pixel_idcs[sample_idx].numpy().astype(int)

            # swap if needed
            if row_start > row_end:
                row_start, row_end = swap(row_start, row_end)
            if col_start > col_end:
                col_start, col_end = swap(col_start, col_end)
            
            paint_it_black = [self.num_pixels * r + c \
                    for r in range(min(row_start, self.num_pixels-1), min(row_end+1, self.num_pixels)) \
                    for c in range(min(col_start, self.num_pixels-1), min(col_end+1, self.num_pixels))]
            
            Y[sample_idx, paint_it_black] = torch.ones(1, len(paint_it_black))

        return Y.to(torch.double)


# utility function

class AreaUtil(torch.nn.Module):
    def __init__(self, weights: Optional[Tensor] = None):
        r"""
        Args:
            weights: `1 x outcome_dim` tensor 
        """
        super().__init__()
        self.weights = weights
    
    def forward(self, Y: Tensor):
        area = torch.sum(Y, dim = 1)
        if self.weights is not None:
            area = area * self.weights
        return area


In [4]:
class ModifiedTransformedPosterior(TransformedPosterior):
    @property
    def event_shape(self) -> torch.Size:
        r"""The event shape (i.e. the shape of a single sample)."""
        return self.rsample().shape[-2:]

    def _extended_shape(
        self, sample_shape: torch.Size = torch.Size()  # noqa: B008
    ) -> torch.Size:
        r"""Returns the shape of the samples produced by the posterior with
        the given `sample_shape`.

        NOTE: This assumes that the `sample_transform` does not change the
        shape of the samples.
        """

        return self.rsample().shape[-2:]

class PCAOutcomeTransform(OutcomeTransform):
    def __init__(
        self,
        variance_explained_threshold: float = 0.9,
        num_axes: Optional[int] = None,
        *tkwargs,
    ):
        r"""
        Initialize PCAOutcomeTransform() instance
        Args:
            variance_explained_threshold: fraction of variance in the data that we want the selected principal axes to explain;
                if num_axes is None, use this to decide the number of principal axes to select
            num_axes: number of principal axes to select
        """

        super().__init__()
        self.variance_explained_threshold = variance_explained_threshold
        self.num_axes = num_axes

    def forward(
        self, Y: torch.Tensor, Yvar: Optional[torch.Tensor] = None, **tkwargs
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        r"""
        Perform PCA on data Y and transforms it to a lower-dimensional representation in terms of principal components.
        Args:
            Y: `batch_shape x num_samples x output_dim` tensor of metric observations;
                assume that it is centered already, since we would normally chain the PCAOutcomeTransform() after a Standardize()
            Yvar: (optional) `batch_shape x num_samples x output_dim` tensor of metric noises (variance)
        Returns:
            Y_transformed: `batch_shape x num_samples x PC_dim` tensor of PC values
            Yvar_transformed: `batch_shape x num_samples x PC_dim` tensor of estimated PC variances
        """

        if self.training:

            U, S, V = torch.svd(Y)
            S_squared = torch.square(S)
            explained_variance = S_squared / S_squared.sum()

            if self.num_axes is None:
                # decide the number of principal axes to keep (that makes explained variance exceed the specified threshold)
                exceed_thres = (
                    np.cumsum(explained_variance) > self.variance_explained_threshold
                )
                self.num_axes = len(exceed_thres) - sum(exceed_thres) + 1

            axes_learned = torch.transpose(V[:, : self.num_axes], -2, -1)
            self.PCA_explained_variance = sum(explained_variance[: self.num_axes])
            self.axes_learned = torch.tensor(axes_learned, **tkwargs)

        Y_transformed = torch.matmul(Y, torch.transpose(self.axes_learned, -2, -1)).to(
            **tkwargs
        )

        # if Yvar is given, the variance of PCs is lower bounded by the linear combination of Yvar terms
        # technically, the variance of PCs should include the covariance between Y's, but that is usually not available
        if Yvar is not None:
            Yvar_transformed = torch.matmul(
                Yvar, torch.square(torch.transpose(self.axes_learned, -2, -1))
            ).to(**tkwargs)

        return Y_transformed, Yvar_transformed if Yvar is not None else None

    def untransform(
        self, Y: torch.Tensor, Yvar: Optional[torch.Tensor] = None, **tkwargs
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        r"""
        Transform PC back to metrics according to self.axes_learned.
        Args:
            Y: `num_samples x PC_dim` tensor of PC values
            Yvar: `num_samples x PC_dim` tensor of PC variances
        Returns:
            Y_untransformed: `num_samples x output_dim` tensor of metric values
            Yvar_untransformed: `num_samples x output_dim` tensor of metric variances
        """

        Y_untransformed = torch.matmul(Y, self.axes_learned)
        if Yvar is not None:
            Yvar_untransformed = torch.matmul(Yvar, torch.square(self.axes_learned))

        return (
            Y_untransformed,
            Yvar_untransformed if Yvar is not None else None,
        )

    def untransform_posterior(self, posterior: Posterior):
        r"""
        Create posterior distribution in the space of metrics.
        Args:
            posterior: posterior in the space of PCs
        Returns:
            untransformed_posterior: posterior in the space of metrics
        """

        untransformed_posterior = ModifiedTransformedPosterior(
            posterior=posterior,
            sample_transform=lambda x: x.matmul(self.axes_learned),
            mean_transform=lambda x, v: x.matmul(self.axes_learned),
            variance_transform=lambda x, v: v.matmul(torch.square(self.axes_learned)),
        )

        return untransformed_posterior


class PCAInputTransform(InputTransform, torch.nn.Module):
    def __init__(
        self,
        axes: torch.Tensor,
        transform_on_train: bool = True,
        transform_on_eval: bool = True,
        transform_on_fantasize: bool = True,
    ):
        r"""
        Initialize PCAInputTransform() instance.
        Args:
            axes: `num_axes x input_dim` tensor with norm-1 orthogonal rows
                (in the case of PE, these are the principal axes
                learned from the previous stage of fitting outcome model)
            transform_on_train: A boolean indicating whether to apply the
                transform in train() mode.
            transform_on_eval: A boolean indicating whether to apply the
                transform in eval() mode.
            transform_on_fantasize: A boolean indicating whether to apply
                the transform when called from within a `fantasize` call.
        """
        super().__init__()
        self.axes = axes
        self.transform_on_train = transform_on_train
        self.transform_on_eval = transform_on_eval
        self.transform_on_fantasize = transform_on_fantasize

    def transform(self, X: torch.Tensor) -> torch.Tensor:
        r"""
        Transform the input X into latent representations using self.axes.
        Args:
            X: `num_samples x input_dim` tensor of input data
        """

        transformed_X = torch.matmul(X, torch.transpose(self.axes, -2, -1))

        return transformed_X

    def untransform(self, X_tf: torch.Tensor) -> torch.Tensor:
        r"""
        Untransform a latent representation back to input space.
        Args:
            X_tf: `num_samples x num_axes` tensor of latent representations
        """

        untransformed_X = torch.matmul(X_tf, self.axes)

        return untransformed_X

In [5]:
from gpytorch.priors.torch_priors import GammaPrior
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.priors import SmoothedBoxPrior
from gpytorch.constraints import GreaterThan, Interval
from botorch.exceptions.errors import BotorchTensorDimensionError

from botorch.models.transforms.input import InputTransform, ReversibleInputTransform


# modified kernel with change in hyperpriors
def make_modified_kernel(ard_num_dims, a=0.2, b=5.0):
    ls_prior = GammaPrior(1.2, 0.5)
    ls_prior_mode = (ls_prior.concentration - 1) / ls_prior.rate

    covar_module = ScaleKernel(
        RBFKernel(
            ard_num_dims=ard_num_dims,
            lengthscale_prior=ls_prior,
            lengthscale_constraint=GreaterThan(
                lower_bound=1e-4, transform=None, initial_value=ls_prior_mode
            ),
        ),
        outputscale_prior=SmoothedBoxPrior(a=a, b=b),
        outputscale_constraint=Interval(lower_bound=a, upper_bound=b),
        # outputscale_prior=SmoothedBoxPrior(a=1e-2, b=1e2),
        # outputscale_prior=SmoothedBoxPrior(a=0.1, b=10),
        # outputscale_constraint=Interval(lower_bound=0.1, upper_bound=10),
    )

    return covar_module




class InputCenter(ReversibleInputTransform, torch.nn.Module):
    r"""Center the inputs (zero mean), don't change the variance.
    This class is modified from InputStandardize.

    In train mode, calling `forward` updates the module state
    (i.e. the mean/std normalizing constants). If in eval mode, calling `forward`
    simply applies the standardization using the current module state.
    """

    def __init__(
        self,
        d: int,
        indices: Optional[List[int]] = None,
        batch_shape: torch.Size = torch.Size(),  # noqa: B008
        transform_on_train: bool = True,
        transform_on_eval: bool = True,
        transform_on_fantasize: bool = True,
        reverse: bool = False,
        min_std: float = 1e-8,
    ) -> None:
        r"""Center inputs (zero mean).

        Args:
            d: The dimension of the input space.
            indices: The indices of the inputs to standardize. If omitted,
                take all dimensions of the inputs into account.
            batch_shape: The batch shape of the inputs (asssuming input tensors
                of shape `batch_shape x n x d`). If provided, perform individual
                normalization per batch, otherwise uses a single normalization.
            transform_on_train: A boolean indicating whether to apply the
                transforms in train() mode. Default: True
            transform_on_eval: A boolean indicating whether to apply the
                transform in eval() mode. Default: True
            reverse: A boolean indicating whether the forward pass should untransform
                the inputs.
            min_std: Amount of noise to add to the standard deviation to ensure no
                division by zero errors.
        """
        super().__init__()
        if (indices is not None) and (len(indices) == 0):
            raise ValueError("`indices` list is empty!")
        if (indices is not None) and (len(indices) > 0):
            indices = torch.tensor(indices, dtype=torch.long)
            if len(indices) > d:
                raise ValueError("Can provide at most `d` indices!")
            if (indices > d - 1).any():
                raise ValueError("Elements of `indices` have to be smaller than `d`!")
            if len(indices.unique()) != len(indices):
                raise ValueError("Elements of `indices` tensor must be unique!")
            self.indices = indices
        self.register_buffer("means", torch.zeros(*batch_shape, 1, d))
        self.register_buffer("stds", torch.ones(*batch_shape, 1, d))
        self._d = d
        self.transform_on_train = transform_on_train
        self.transform_on_eval = transform_on_eval
        self.transform_on_fantasize = transform_on_fantasize
        self.batch_shape = batch_shape
        self.min_std = min_std
        self.reverse = reverse
        self.learn_bounds = True

    def _transform(self, X: Tensor) -> Tensor:
        r"""Center the inputs.

        In train mode, calling `forward` updates the module state
        (i.e. the mean/std normalizing constants). If in eval mode, calling `forward`
        simply applies the standardization using the current module state.

        Args:
            X: A `batch_shape x n x d`-dim tensor of inputs.

        Returns:
            A `batch_shape x n x d`-dim tensor of inputs, de-meaned.
        """
        if self.training and self.learn_bounds:
            if X.size(-1) != self.means.size(-1):
                raise BotorchTensorDimensionError(
                    f"Wrong input. dimension. Received {X.size(-1)}, "
                    f"expected {self.means.size(-1)}"
                )

            n = len(self.batch_shape) + 2
            if X.ndim < n:
                raise ValueError(
                    f"`X` must have at least {n} dimensions, {n - 2} batch and 2 innate"
                    f" , but has {X.ndim}."
                )

            # Aggregate means and standard deviations over extra batch and marginal dims
            batch_ndim = min(len(self.batch_shape), X.ndim - 2)  # batch rank of `X`
            reduce_dims = (*range(X.ndim - batch_ndim - 2), X.ndim - 2)
            self.stds, self.means = (
                values.unsqueeze(-2)
                for values in torch.std_mean(X, dim=reduce_dims, unbiased=True)
            )
            self.stds.clamp_(min=self.min_std)

        if hasattr(self, "indices"):
            X_new = X.clone()
            X_new[..., self.indices] = (
                X_new[..., self.indices] - self.means[..., self.indices]
            )

            return X_new

        return X - self.means

    def _untransform(self, X: Tensor) -> Tensor:
        r"""Un-center the inputs, i.e., add back the mean.

        Args:
            X: A `batch_shape x n x d`-dim tensor of centered inputs.

        Returns:
            A `batch_shape x n x d`-dim tensor of un-centered inputs.
        """
        if hasattr(self, "indices"):
            X_new = X.clone()
            X_new[..., self.indices] = (
                self.means[..., self.indices] + X_new[..., self.indices]
            )

            return X_new

        return self.means.to(X) + X

    def equals(self, other: InputTransform) -> bool:
        r"""Check if another input transform is equivalent.

        Args:
            other: Another input transform.

        Returns:
            A boolean indicating if the other transform is equivalent.
        """
        if hasattr(self, "indices") == hasattr(other, "indices"):
            if hasattr(self, "indices"):
                return (
                    super().equals(other=other)
                    and (self._d == other._d)
                    and (self.indices == other.indices).all()
                )
            else:
                return super().equals(other=other) and (self._d == other._d)
        return False


In [13]:
from botorch.acquisition.objective import MCAcquisitionObjective
from botorch.models.model import Model
from botorch.acquisition.monte_carlo import (qNoisyExpectedImprovement,
                                             qSimpleRegret)

def generate_random_inputs(problem: torch.nn.Module, n: int) -> Tensor:
    r"""Generate n quasi-random Sobol points in the design space.
    Args:
        problem: a TestProblem in Botorch
        n: number of random inputs to generate
    Returns:
        `n x problem input dim` tensor of randomly generated points in problem's input domain
    """
    return (
        draw_sobol_samples(bounds=problem.bounds, n=1, q=n).squeeze(0).to(torch.double)
    )


def gen_comps(
    util_vals: Tensor, comp_noise_type: str = None, comp_noise: float = None
) -> Tensor:
    r"""Create pairwise comparisons.
    Args:
        util_vals: `num_outcomes x 1` tensor of utility values
        comp_noise_type: type of comparison noise to inject, one of {'constant', 'probit'}
        comp_noise: parameter related to probability of making a comparison mistake
    Returns:
        comp_pairs: `(num_outcomes // 2) x 2` tensor showing the preference,
            with the more preferable outcome followed by the other one in each row
    """
    cpu_util = util_vals.cpu()

    comp_pairs = []
    for i in range(cpu_util.shape[0] // 2):
        i1 = i * 2
        i2 = i * 2 + 1
        if cpu_util[i1] > cpu_util[i2]:
            new_comp = [i1, i2]
            util_diff = cpu_util[i1] - cpu_util[i2]
        else:
            new_comp = [i2, i1]
            util_diff = cpu_util[i2] - cpu_util[i1]

        new_comp = torch.tensor(new_comp, device=util_vals.device, dtype=torch.long)
        # if comp_noise_type is not None:
        #     new_comp = inject_comp_error(
        #         new_comp, util_diff, comp_noise_type, comp_noise
        #     )
        comp_pairs.append(new_comp)

    comp_pairs = torch.stack(comp_pairs)

    return comp_pairs


def gen_initial_real_data(
    n: int, problem: torch.nn.Module, util_func: torch.nn.Module, comp_noise: float = 0, batch_eval: bool = True
) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
    # generate (noisy) ground truth data
    r"""Generate noisy ground truth inputs, outcomes, utility values, and comparisons.
    Args:
        n: number of samples to generate
        problem: a TestProblem
        util_func: ground truth utility function (outcome -> utility)
        comp_noise: noise to inject into the comparisons
    Returns:
        X: generated inputs
        Y: generated (noisy) outcomes from evaluating the problem on X
        util_vals: utility values of generated Y
        comps: comparison results for adjacent pairs of util_vals
    """

    X = generate_random_inputs(problem, n).detach()

    if not batch_eval:
        Y_list = []
        for idx in range(len(X)):
            y = problem(X[idx]).detach()
            Y_list.append(y)
        Y = torch.stack(Y_list).squeeze(1)
    else:
        Y = problem(X).detach()

    util_vals = util_func(Y).detach()
    # comps = gen_comps(util_vals, comp_noise_type="constant", comp_noise=comp_noise)
    comps = gen_comps(util_vals)

    return X, Y, util_vals, comps



def find_true_optimal_utility(
    problem: torch.nn.Module, 
    util_func: torch.nn.Module, 
    n: int,
    maximize: bool = True
):
    r"""
    Find the optimal utility value, i.e., max_x (util_func(problem(x))) across
    the domain through taking a large number of samples.
    Args:
        problem: a TestProblem that maps designs to outcomes
        util_func: maps outcomes to scalar utility
        n: number of evalutions
        maximize: boolean for whether to maximize (if False, minimize)
    """

    meta_batch_size = 20000 // problem.dim
    num_meta_batches = n // meta_batch_size + 1
    best_util_vals = []

    for _ in range(num_meta_batches):
        _, _, util_vals, _ = gen_initial_real_data(
            n=meta_batch_size,
            problem=problem,
            util_func=util_func,
            comp_noise=0,
            batch_eval=True
        )
        if maximize:
            best_util_vals.append(torch.max(util_vals).item())
        else:
            best_util_vals.append(torch.min(util_vals).item())
    
    if maximize:
        return np.max(best_util_vals)
    else:
        return np.min(best_util_vals)



def gen_exp_cand(
    outcome_model: Model,
    objective: MCAcquisitionObjective,
    problem: torch.nn.Module,
    q: int,
    acqf_name: str,
    seed: int,
    X: Optional[Tensor] = None,
    sampler_num_outcome_samples: int = 64,
    num_restarts: int = 8,
    raw_samples: int = 64,
    batch_limit: int = 4,
) -> Tensor:
    """Given an outcome model and an objective, generate q experimental candidates
    using a specified acquisition function.
    Args:
        outcome_model: GP model mapping input to outcome
        objective: MC objective mapping outcome to utility
        problem: a TestProblem
        q: number of candidates to generate
        acqf_name: name of acquisition function, one of {'qNEI', 'posterior_mean'}
        X: `num_outcome_samples x input_dim` current training data
        sampler_num_outcome_samples: number of base samples in acq function's sampler
        num_restarts: number of starting points for multi-start acqf optimization
        raw_samples: number of samples for initializing acqf optimization
        batch_limit: the limit on batch size in gen_candidates_scipy() within optimize_acqf()
    Returns:
        candidates: `q x problem input dim` generated candidates
    """
    sampler = SobolQMCNormalSampler(sampler_num_outcome_samples)
    if acqf_name == "qNEI":
        # generate experimental candidates with qNEI/qNEIUU
        acq_func = qNoisyExpectedImprovement(
            model=outcome_model,
            objective=objective,
            X_baseline=X,
            sampler=sampler,
            prune_baseline=True,
            cache_root=False,
        )
    elif acqf_name == "posterior_mean":
        # generate experimental candidates with maximum posterior mean
        acq_func = qSimpleRegret(
            model=outcome_model,
            sampler=sampler,
            objective=objective,
        )
    else:
        raise RuntimeError("Unknown acquisition function name!")

    # optimize the acquisition function
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        q=q,
        bounds=problem.bounds,
        num_restarts=num_restarts,
        raw_samples=raw_samples,
        options={"batch_limit": batch_limit, "seed": seed},
        sequential=True,
    )
    return candidates

In [7]:
from botorch.models.model import Model
from botorch.models.deterministic import DeterministicModel


class ModifiedFixedSingleSampleModel(DeterministicModel):
    r"""
    A deterministic model defined by a single sample `w`.

    Given a base model `f` and a fixed sample `w`, the model always outputs

        y = f_mean(x) + f_stddev(x) * w

    We assume the outcomes are uncorrelated here.

    This is modified from FixedSingleSampleModel to handle dimensionality reduction.
    For models with dim reduction, model.num_outputs is the reduced outcome dimension,
    whereas we want w to be in the original outcome dimension.
    In this modification, we define self.w within forward() rather than __init__(),
    where we fix the dimensionality of w to be posterior(X).event_shape[-1].
    """

    def __init__(
        self, model: Model, outcome_dim: int, w: Optional[torch.Tensor] = None
    ) -> None:
        r"""
        Args:
            model: The base model.
            outcome_dim: dimensionality of the outcome space
            w: A 1-d tensor with length = outcome_dim.
                If None, draw it from a standard normal distribution.
        """
        super().__init__()
        self.model = model
        self.w = torch.randn(outcome_dim)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        post = self.model.posterior(X)

        return post.mean + post.variance.sqrt() * self.w.to(X)


In [8]:
from botorch.models import SingleTaskGP

def fit_outcome_model(X: torch.Tensor, Y: torch.Tensor, **model_kwargs) -> Model:
    r"""Fit outcome model.
    Args:
        X: `num_samples x input_dim` input data
        Y: `num_samples x outcome_dim` outcome data
        model_kwargs: arguments for fitting outcome GP,
            such as outcome_transform, covar_module, likelihood, etc.
    Returns:
        outcome_model: Fitted outcome model mapping input to outcome
    """

    outcome_model = SingleTaskGP(train_X=X, train_Y=Y, **model_kwargs)

    mll_outcome = ExactMarginalLogLikelihood(outcome_model.likelihood, outcome_model)
    # fit_gpytorch_model(mll_outcome)
    fit_gpytorch_mll(mll_outcome)

    return outcome_model

In [15]:
# BOPE code


class BopeExperiment:

    attr_list = {
        "pca_var_threshold": 0.95,
        "initial_experimentation_batch": 16,
        "n_check_post_mean": 20,
        "every_n_comps": 3,
        "verbose": True,
        "dtype": torch.double,
        "noise_std": 0.01,  # TODO: figure out how to set this for different probs
        "num_restarts": 20,
        "raw_samples": 128,
        "batch_limit": 4,
        "sampler_num_outcome_samples": 64,
        "maxiter": 1000,
        "latent_dim": None,
        "min_stdv": 100000,
        "true_axes": None, # specify these for synthetic problems
        "save": False
    }

    def __init__(
        self,
        problem: torch.nn.Module,
        util_func: torch.nn.Module,
        methods: List[str],
        pe_strategies: List[str],
        trial_idx: int,
        output_path: Optional[str]=None,
        save: bool =False,
        **kwargs
    ) -> None:
        """
        specify experiment settings
            problem:
            util_func:
            methods: list of statistical methods, each denoted using a str
            pe_strategies: PE strategies to use, could be {"EUBO-zeta", "Random-f"}
        one run should handle one problem and >=1 methods and >=1 pe_strategies
        """

        # self.attr_list stores default values, then overwrite with kwargs
        for key in self.attr_list.keys():
            setattr(self, key, self.attr_list[key])
        for key in kwargs.keys():
            setattr(self, key, kwargs[key])
        
        print("BopeExperiment settings: ", self.attr_list)

        # pre-specified experiment metadata
        self.problem = problem
        self.util_func = util_func
        self.pe_strategies = pe_strategies
        self.outcome_dim = problem.outcome_dim
        self.input_dim = problem._bounds.shape[-1]
        self.trial_idx = trial_idx
        self.output_path = output_path
        if save:
            if not os.path.exists(self.output_path):
                os.makedirs(self.output_path)
        if hasattr(self.problem, "true_axes"):
            self.true_axes = self.problem.true_axes

        if "pca" in methods:
            # make sure to run pca first, so that the learned latent_dim
            # informs the other methods with dim reduction
            self.methods = ["pca"] + [m for m in methods if m != "pca"]
            print('self.methods, ', self.methods)
        else:
            # if "pca" is not run and latent_dim is not specified
            # set self.latent_dim by hand
            self.methods = methods
            if not self.latent_dim:
                self.latent_dim = self.outcome_dim // 3

        # logging models and results
        self.outcome_models_dict = {}  # by method
        self.pref_data_dict = defaultdict(dict)  # by (method, pe_strategy)
        self.PE_time_dict = defaultdict(dict)
        self.PE_session_results = defaultdict(dict) # deps on method and pe strategy
        self.final_candidate_results = defaultdict(dict) # deps on method and pe strategy
        self.outcome_model_fitting_results = defaultdict(dict) # only deps on method

        # specify outcome and input transforms, covariance modules
        self.transforms_covar_dict = {
            "st": {
                "outcome_tf": Standardize(self.outcome_dim),
                "input_tf": Normalize(self.outcome_dim),
                "covar_module": make_modified_kernel(ard_num_dims=self.outcome_dim),
            },
            "pca": {
                "outcome_tf": ChainedOutcomeTransform(
                    **{
                        "standardize": Standardize(
                            self.outcome_dim,
                            min_stdv=self.min_stdv,
                        ),
                        "pca": PCAOutcomeTransform(
                            variance_explained_threshold=self.pca_var_threshold,
                            num_axes=self.latent_dim,
                        ),
                    }
                ),
            },            
        }
        
        # compute true optimal utility
        self.true_opt = find_true_optimal_utility(self.problem, self.util_func, n=5000)

    def generate_random_experiment_data(self, n, compute_util: False):
        r"""Generate n observations of experimental designs and outcomes.
        Args:
            problem: a TestProblem
            n: number of samples
        Computes:
            X: `n x problem input dim` tensor of sampled inputs
            Y: `n x problem outcome dim` tensor of noisy evaluated outcomes at X
            util_vals: `n x 1` tensor of utility value of Y
            comps: `n/2 x 2` tensor of pairwise comparisons 
        """

        self.X = (
            draw_sobol_samples(bounds=self.problem.bounds, n=1, q=n)
            .squeeze(0)
            .to(torch.double)
            .detach()
        )
        self.Y = self.problem(self.X).detach()

        if compute_util:
            self.util_vals = self.util_func(self.Y).detach()
            self.comps = gen_comps(self.util_vals)

    def fit_outcome_model(self, method):
        r"""Fit outcome model based on specified method.
        Args:
            method: string specifying the statistical model
        """
        print(f"Fitting outcome model using {method}")
        start_time = time.time()

        outcome_model = fit_outcome_model(
            self.X,
            self.Y,
            outcome_transform=self.transforms_covar_dict[method]["outcome_tf"],
        )
        
        model_fitting_time = time.time() - start_time
        # rel_mse = check_outcome_model_fit(outcome_model, self.problem, n_test=1000)
        # self.outcome_model_fitting_results[method] = {
        #     "model_fitting_time": model_fitting_time,
        #     "rel_mse": rel_mse
        # }

        if method == "pca":
            self.pca_axes = outcome_model.outcome_transform["pca"].axes_learned
            self.transforms_covar_dict["pca"]["input_tf"] = ChainedInputTransform(
                **{
                    # "standardize": InputStandardize(config["outcome_dim"]),
                    # TODO: was trying standardize again
                    "center": InputCenter(self.outcome_dim),
                    "pca": PCAInputTransform(axes=self.pca_axes),
                }
            )

            self.latent_dim = self.pca_axes.shape[0]
            print(
                f"amount of variance explained by {self.latent_dim} axes: {outcome_model.outcome_transform['pca'].PCA_explained_variance}"
            )
            self.outcome_model_fitting_results[method].update(
                {"num_pca_axes": self.latent_dim}
            )

            # here we first see how many latent dimensions PCA learn
            # then create a random linear proj onto the same dimensionality
            # similarly, set that as the cardinality of the random subset 
            self.transforms_covar_dict["pca"]["covar_module"] = make_modified_kernel(
                ard_num_dims=self.latent_dim
            )

        self.outcome_models_dict[method] = outcome_model

    def fit_pref_model(
        self,
        Y,
        comps,
        **model_kwargs
    ):
        r"""Fit utility model based on given data and model_kwargs
        Args:
            Y: `num_samples x outcome_dim` tensor of outcomes
            comps: `num_samples/2 x 2` tensor of pairwise comparisons of Y data
            model_kwargs: input transform and covar_module
        """
        util_model = PairwiseGP(
            datapoints=Y, comparisons=comps, **model_kwargs)

        mll_util = PairwiseLaplaceMarginalLogLikelihood(
            util_model.likelihood, util_model)
        fit_gpytorch_mll(mll_util)

        return util_model

    def run_pref_learning(self, method, pe_strategy):

        acqf_vals = []
        for i in range(self.every_n_comps):

            train_Y, train_comps = self.pref_data_dict[method][pe_strategy]

            print(
                f"Running {i+1}/{self.every_n_comps} preference learning using {pe_strategy}")
            print("train_Y, train_comps shapes: ", train_Y.shape, train_comps.shape)

            fit_model_succeed = False

            for _ in range(3):
                try:
                    pref_model = self.fit_pref_model(
                        train_Y,
                        train_comps,
                        input_transform=self.transforms_covar_dict[method]["input_tf"],
                        covar_module=self.transforms_covar_dict[method]["covar_module"],
                        # likelihood=likelihood,
                    )
                    print("Pref model fitting successful")
                    fit_model_succeed = True
                    break
                except (ValueError, RuntimeError):
                    continue
            if not fit_model_succeed:
                print(
                    "fit_pref_model() failed 3 times, stop current call of run_pref_learn()"
                )

            if pe_strategy == "EUBO-zeta":
                with botorch.settings.debug(state=True):
                    # EUBO-zeta
                    one_sample_outcome_model = ModifiedFixedSingleSampleModel(
                        model=self.outcome_models_dict[method],
                        outcome_dim=train_Y.shape[-1]
                    ).to(torch.double) # TODO: just added this but didn't help
                    acqf = AnalyticExpectedUtilityOfBestOption(
                        pref_model=pref_model,
                        outcome_model=one_sample_outcome_model
                    ).to(torch.double)  # TODO: just added this but didn't help
                    found_valid_candidate = False
                    for _ in range(3):
                        try:
                            cand_X, acqf_val = optimize_acqf(
                                acq_function=acqf,
                                q=2,
                                bounds=self.problem.bounds,
                                num_restarts=self.num_restarts,
                                raw_samples=self.raw_samples,  # used for intialization heuristic
                                options={"batch_limit": 4, "seed": self.trial_idx},
                            )
                            cand_Y = one_sample_outcome_model(cand_X)
                            acqf_vals.append(acqf_val.item())

                            found_valid_candidate = True
                            break
                        except (ValueError, RuntimeError) as error:
                            print("error in optimizing EUBO: ", error)
                            continue
                    if not found_valid_candidate:
                        print(
                            f"optimize_acqf() failed 3 times for EUBO with {method},", 
                            "stop current call of run_pref_learn()"
                        )
                        print("Current train_Y, train_comps: ", train_Y, train_comps)
                        return

            elif pe_strategy == "Random-f":
                # Random-f
                cand_X = draw_sobol_samples(
                    bounds=self.problem.bounds,
                    n=1,
                    q=2,
                ).squeeze(0).to(torch.double)
                cand_Y = self.outcome_models_dict[method].posterior(
                    cand_X).rsample().squeeze(0).detach()
            else:
                raise RuntimeError("Unknown preference exploration strategy!")

            cand_Y = cand_Y.detach().clone()
            cand_comps = gen_comps(self.util_func(cand_Y))

            train_comps = torch.cat(
                (train_comps, cand_comps + train_Y.shape[0])
            )
            train_Y = torch.cat((train_Y, cand_Y))
            print('train_Y, train_comps shape: ', train_Y.shape, train_comps.shape)

            self.pref_data_dict[method][pe_strategy] = (train_Y, train_comps)


    def find_max_posterior_mean(self, method, pe_strategy, num_pref_samples=1):

        train_Y, train_comps = self.pref_data_dict[method][pe_strategy]

        within_result = {}

        pref_model = self.fit_pref_model(
            Y=train_Y,
            comps=train_comps,
            input_transform=self.transforms_covar_dict[method]["input_tf"],
            covar_module=self.transforms_covar_dict[method]["covar_module"]
        )
        sampler = SobolQMCNormalSampler(num_pref_samples)
        pref_obj = LearnedObjective(pref_model=pref_model, sampler=sampler)

        # find experimental candidate(s) that maximize the posterior mean utility
        post_mean_cand_X = gen_exp_cand(
            outcome_model=self.outcome_models_dict[method],
            objective=pref_obj,
            problem=self.problem,
            q=1,
            acqf_name="posterior_mean",
            seed=self.trial_idx
        )

        post_mean_util = self.util_func(
            self.problem.evaluate_true(post_mean_cand_X)).item()
        print(
            f"True utility of the current posterior mean utility maximizing design: {post_mean_util:.3f}")

        within_result = {
            "n_comps": train_comps.shape[0],
            "util": post_mean_util,
            "run_id": self.trial_idx,
            "pe_strategy": pe_strategy,
            "method": method,
        }

        return within_result

    def generate_final_candidate(self, method, pe_strategy):

        train_Y, train_comps = self.pref_data_dict[method][pe_strategy]

        pref_model = self.fit_pref_model(
            Y=train_Y,
            comps=train_comps,
            input_transform=self.transforms_covar_dict[method]["input_tf"],
            covar_module=self.transforms_covar_dict[method]["covar_module"]
        )

        sampler = SobolQMCNormalSampler(1)
        pref_obj = LearnedObjective(pref_model=pref_model, sampler=sampler)

        # find experimental candidate(s) that maximize the posterior mean utility
        cand_X = gen_exp_cand(
            outcome_model=self.outcome_models_dict[method],
            objective=pref_obj,
            problem=self.problem,
            q=1,
            acqf_name="qNEI",
            X=self.X,
            seed=self.trial_idx
        )

        qneiuu_util = self.util_func(self.problem.evaluate_true(cand_X)).item()
        print(
            f"{method}-{pe_strategy} qNEIUU candidate utility: {qneiuu_util:.5f}"
        )

        exp_result = {
            "candidate": cand_X,
            "candidate_util": qneiuu_util,
            "method": method,
            "strategy": pe_strategy,
            "run_id": self.trial_idx,
            "PE_time": self.PE_time_dict[method][pe_strategy],
            # "util_model_acc": util_model_acc,
        }
        
        # log the true optimal utility computed in __init__()
        exp_result["true_opt"] = self.true_opt

        self.final_candidate_results[method][pe_strategy] = exp_result

    def generate_random_pref_data(self, method, n):

        X = (
            draw_sobol_samples(
                bounds=self.problem.bounds,
                n=1,
                q=2*n,
                seed=self.trial_idx 
            )
            .squeeze(0)
            .to(torch.double)
            .detach()
        )
        Y = self.outcome_models_dict[method].posterior(
            X).rsample().squeeze(0).detach()
        util = self.util_func(Y)
        comps = gen_comps(util)

        for pe_strategy in self.pe_strategies:
            self.pref_data_dict[method][pe_strategy] = (Y, comps)


# ======== Putting it together into steps ========


    def run_first_experimentation_stage(self, method):

        self.fit_outcome_model(method)


    def run_PE_stage(self, method):
        # initial result stored in self.pref_data_dict
        self.generate_random_pref_data(method, n=1)

        for pe_strategy in self.pe_strategies:

            start_time = time.time()

            print(f"===== Running PE using {method} with {pe_strategy} =====")

            self.PE_session_results[method][pe_strategy] = []

            self.PE_session_results[method][pe_strategy].append(
                self.find_max_posterior_mean(method, pe_strategy)
            )
            for j in range(self.n_check_post_mean):
                self.run_pref_learning(method, pe_strategy)
                self.PE_session_results[method][pe_strategy].append(
                    self.find_max_posterior_mean(method, pe_strategy)
                )
            
            # log time required to do PE
            PE_time = time.time() - start_time
            self.PE_time_dict[method][pe_strategy] = PE_time # will be logged later


    def run_second_experimentation_stage(self, method):
        for pe_strategy in self.pe_strategies:
            print(f"===== Generating final candidate using {method} with {pe_strategy} =====")
            self.generate_final_candidate(method, pe_strategy)


# ======== BOPE loop ========


    def run_BOPE_loop(self):
        # handle multiple trials
        # have a flag for whether the fitting is successful or not

        # all methods use the same initial experimentation data
        self.generate_random_experiment_data(
            self.initial_experimentation_batch,
            compute_util=True
        )

        for method in self.methods:
            print(f"============= Running {method} =============")
            self.run_first_experimentation_stage(method)
            self.run_PE_stage(method)
            self.run_second_experimentation_stage(method)

            if self.save:

                torch.save(self.PE_session_results, self.output_path +
                        'PE_session_results_trial=' + str(self.trial_idx) + '.th')
                torch.save(self.final_candidate_results, self.output_path +
                        'final_candidate_results_trial=' + str(self.trial_idx) + '.th')
                torch.save(self.outcome_model_fitting_results, self.output_path +
                        'outcome_model_fitting_results_trial=' + str(self.trial_idx) + '.th')



In [16]:
torch.manual_seed(0)

problem = Image(num_pixels=8)
util_func = AreaUtil()

experiment = BopeExperiment(
    problem, 
    util_func, 
    methods = ["st", "pca"],
    pe_strategies = ["EUBO-zeta", "Random-f"],
    trial_idx = 0)

experiment.run_BOPE_loop()

/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/botorch/test_functions/base.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "bounds", torch.tensor(self._bounds, dtype=torch.float).transpose(-1, -2)


BopeExperiment settings:  {'pca_var_threshold': 0.95, 'initial_experimentation_batch': 16, 'n_check_post_mean': 20, 'every_n_comps': 3, 'verbose': True, 'dtype': torch.float64, 'noise_std': 0.01, 'num_restarts': 20, 'raw_samples': 128, 'batch_limit': 4, 'sampler_num_outcome_samples': 64, 'maxiter': 1000, 'latent_dim': None, 'min_stdv': 100000, 'true_axes': None, 'save': False}
self.methods,  ['pca', 'st']
============= Running pca =============
Fitting outcome model using pca


/tmp/ipykernel_3146110/1961457152.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.axes_learned = torch.tensor(axes_learned, **tkwargs)


amount of variance explained by 10 axes: 0.9546244571245466
===== Running PE using pca with EUBO-zeta =====
True utility of the current posterior mean utility maximizing design: 4.000
Running 1/3 preference learning using EUBO-zeta
train_Y, train_comps shapes:  torch.Size([2, 64]) torch.Size([1, 2])
Pref model fitting successful
error in optimizing EUBO:  32 elements of the 32 element gradient array `gradf` are NaN. This often indicates numerical issues. Consider using `dtype=torch.double`.
error in optimizing EUBO:  32 elements of the 32 element gradient array `gradf` are NaN. This often indicates numerical issues. Consider using `dtype=torch.double`.
error in optimizing EUBO:  32 elements of the 32 element gradient array `gradf` are NaN. This often indicates numerical issues. Consider using `dtype=torch.double`.
optimize_acqf() failed 3 times for EUBO with pca, stop current call of run_pref_learn()
Current train_Y, train_comps:  tensor([[-5.6462e-16,  8.1149e-16,  2.4462e-01,  2.4462

KeyboardInterrupt: 